In [ ]:
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import segmentation_models_pytorch as smp

from src.models.BaselineModel import BaselineModel
from src.evaluation.evaluate_result import evaluate_result
from src.datasets.DubaiSemanticSegmentationDataset import (
    DubaiSemanticSegmentationDataset,
)

from src.datasets.utils.ResizeToDivisibleBy32 import ResizeToDivisibleBy32

## Prepare environment

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
VAL_SIZE = 0.2
BATCH_SIZE = 1
SEED = 42
DUBAI_DATASET_PATH = "data/DubaiSemanticSegmentationDataset"

In [ ]:
train_dataset = DubaiSemanticSegmentationDataset(
    DUBAI_DATASET_PATH, transforms=[ResizeToDivisibleBy32()]
)
print(len(train_dataset))

## Sanity check data

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [ ]:
for images, masks in train_loader:
    print(images.shape)
    print(masks.shape)
    break

In [ ]:
to_pil_transform = transforms.ToPILImage()

In [ ]:
img = to_pil_transform(images.squeeze())

In [ ]:
# img.show()

In [ ]:
msk = to_pil_transform(masks.squeeze())

In [ ]:
# msk.show()

## Run example model on single image

In [ ]:
model = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=6,  # model output channels (number of classes in your dataset)
).to(device)

In [ ]:
baseline_model = BaselineModel(classes=6).to(device)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    # output = (output > 0.5).float()

    output_baseline = baseline_model(images.to(device))
    # output_baseline = (output_baseline > 0.5).float()

In [ ]:
print(output.shape)

In [ ]:
print(output_baseline.shape)

In [ ]:
unique, counts = np.unique(output.cpu(), return_counts=True)
print(len(unique))
print(len(counts))

In [ ]:
unique, counts = np.unique(masks.cpu(), return_counts=True)
print(len(unique))
print(len(counts))
print(dict(zip(unique, counts)))

In [ ]:
unique, counts = np.unique(torch.round(masks.cpu(), decimals=3), return_counts=True)
print(len(unique))
print(len(counts))
print(dict(zip(unique, counts)))

In [ ]:
evaluate_result(output, masks)

In [ ]:
evaluate_result(output_baseline, masks)

## Run example model on dataset

In [ ]:
model.eval()
metrics_dict = {
    "iou": [],
    "f1": [],
    "accuracy": [],
    "recall": [],
}

In [ ]:
with torch.no_grad():
    for images, masks in train_loader:
        output = model(images.to(device))
        # output = (output > 0.5).float()

        iter_metrics = evaluate_result(output, masks)
        for key in metrics_dict.keys():
            metrics_dict[key].append(iter_metrics[key])

In [ ]:
print(metrics_dict)